In [1]:
import os
import sys
import pandas as pd
sys.path.append('/Users/dane/src/datatools')

In [2]:
%cd /Users/dane/src/datatools/ipy/notebooks/regtest/dev
df = pd.read_pickle('../data/v14_b8600_b8658.pickle')

/Users/dane/src/datatools/ipy/notebooks/regtest/dev


In [3]:
from regtest.ingest_log_files import StrMap
pmap = StrMap.read('../../../../regtest/project_name')

# build a reverse map and shorten names to useful part
rmap = {v:k.replace('\\', '/')[37:] for k, v in pmap.items()}

In [4]:
df.head(1)

,passed,build,revision,project,machine,sim_time,elapsed_time,os,aut,run_name
0,1,B8622,REV10,0,TESTEXAMPLES,1.284,1.45576,0,153,B8622.REV10


In [5]:
results = []
for project, runs in df.groupby('project'):
    sim_mean = runs.sim_time.mean()
    std = runs.sim_time.std()
    sim_rsd = std / sim_mean
    outliers = len(runs[runs.sim_time > (sim_mean + 2 * std)])
    el_mean = runs.elapsed_time.mean()
    el_rsd = runs.elapsed_time.std() / el_mean
    results.append( (project, sim_mean, sim_rsd, outliers, el_mean, el_rsd) )

test_times = pd.DataFrame.from_records(results,
                                       columns=['project', 'sim_mean', 'sim_rsd',
                                                'outliers', 'el_mean', 'el_rsd'])


In [6]:
# add names to the times
test_times['name'] = test_times.index.map(rmap.get)

In [7]:
# sort by desc rsd for significant sim times
test_times[test_times.sim_mean > 0.5].sort(['sim_rsd'], ascending=[0]).head(20)

,project,sim_mean,sim_rsd,outliers,el_mean,el_rsd,name
7158,7158,1.750333,7.631726,1,2.082314,6.413051,buildtests/InheritedParams/UCT_parameter_passi...
10152,10152,15.019083,2.788927,1,27.707090,2.839702,scripting/API/pcb_import/odb/test_projects/neg...
6451,6451,0.706141,2.426460,1,0.931185,1.840344,buildtests/EM_Tests/SPP_per_layer/tt17188_TTdi...
4929,4929,0.511912,2.232192,1,0.681958,1.675400,buildtests/ALFREDO/MOST/NMOS_VS_PMOS.emp
6679,6679,0.636354,2.071892,1,0.820060,1.616672,buildtests/EXPORT_TRANSFORMS_APLAC/MEXTRAM504/...
4760,4760,1.200650,1.690065,1,1.305103,1.554686,buildtests/ALFREDO/HICUM_L0/TwoStages.emp
6758,6758,1.351933,1.651437,1,1.560647,1.430337,buildtests/Extraction/extraction_groups/Group_...
6520,6520,1.371435,1.057461,1,1.525906,0.950260,buildtests/Examples/60 Examples/Harmonic Balan...
10138,10138,0.677997,0.944866,0,0.746637,0.858222,labview/TT16791.emp
8514,8514,2.955653,0.820674,0,3.158162,0.781051,buildtests/SysBlocks/PHARRAY.emp
